# Part 12: 在加密数据上训练加密神经网络

在此笔记本中，我们将使用到目前为止所学的所有技术来执行神经网络训练（和预测），同时对模型和数据进行加密。

特别是，我们介绍了可用于加密计算的自定义Autograd引擎。

作者:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

中文版译者：
- Hou Wei - github：[@dljgs1](https://github.com/dljgs1)

# 第1步: 创建工作机和玩具数据集

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [ ]:
# 设置好所需
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [ ]:
# 一个玩具数据集
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# 一个玩具模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# 第2步: 对模型和数据加密

加密分为两个步骤。 由于安全多方计算仅适用于整数，因此为了对带小数点的数字（例如权重和激活数）进行运算，我们需要使用固定精度对所有数字进行编码，这将为我们提供几位十进制精度。 我们通过调用.fix_precision()来实现。

然后，我们可以像调用其他演示一样调用.share()，它将通过在Alice和Bob之间共享它们来加密所有值。 请注意，我们还将require_grad设置为True，这也为加密数据添加了一种特殊的autograd方法。 确实，由于安全多方计算不适用于浮点值，因此我们无法使用通常的PyTorch自动分级。因此，我们需要添加一个特殊的AutogradTensor节点来计算梯度图以进行反向传播。您可以打印任何此元素以查看其包含AutogradTensor。

In [ ]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [ ]:
print(data)

# 第3步: 训练

现在我们可以使用简单的张量逻辑进行训练。

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

损失确实减少了！

## 固定精度的影响

您可能想知道加密所有内容如何影响不断减少的损失。实际上，由于理论计算是相同的，因此数字非常接近非加密训练。您可以通过运行相同的示例来验证这一点，而无需加密，并且可以对模型进行确定性的初始化，例如在模型__init__中：

You might wonder how encrypting everything impacts the decreasing loss. Actually, because the theoretical computation is the same, the numbers are very close to non-encrypted training. You can verify this by running the same example without encryption and with a deterministic initialisation of the model like this one in the model `__init__`:
```
with torch.no_grad():
    self.fc1.weight.set_(torch.tensor([[ 0.0738, -0.2109],[-0.1579,  0.3174]], requires_grad=True))
    self.fc1.bias.set_(torch.tensor([0.,0.1], requires_grad=True))
    self.fc2.weight.set_(torch.tensor([[-0.5368,  0.7050]], requires_grad=True))
    self.fc2.bias.set_(torch.tensor([-0.0343], requires_grad=True))
```

您可能会观察到的细微差异是由于在转换为固定精度时对值进行了四舍五入。 默认的`precision_fractional`为3，如果将其降低到2，则明文训练的差异会增加，而如果选择`precision_fractional = 4`，则差异会减小。

# 恭喜!!! 是时候加入社区了!

祝贺您完成本笔记本教程！ 如果您喜欢此方法，并希望加入保护隐私、去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

### 给 PySyft 加星

帮助我们的社区的最简单方法是仅通过给GitHub存储库加注星标！ 这有助于提高人们对我们正在构建的出色工具的认识。

- [Star PySyft](https://github.com/OpenMined/PySyft)


### 加入我们的 Slack!

保持最新进展的最佳方法是加入我们的社区！ 您可以通过填写以下表格来做到这一点[http://slack.openmined.org](http://slack.openmined.org)

### 加入代码项目!

对我们的社区做出贡献的最好方法是成为代码贡献者！ 您随时可以转到PySyft GitHub的Issue页面并过滤“projects”。这将向您显示所有概述，选择您可以加入的项目！如果您不想加入项目，但是想做一些编码，则还可以通过搜索标记为“good first issue”的GitHub问题来寻找更多的“一次性”微型项目。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 捐赠

如果您没有时间为我们的代码库做贡献，但仍想提供支持，那么您也可以成为Open Collective的支持者。所有捐款都将用于我们的网络托管和其他社区支出，例如黑客马拉松和聚会！

[OpenMined's Open Collective Page](https://opencollective.com/openmined)